In [ ]:
# imports
import os, sys, pathlib
root = pathlib.Path.cwd()
# Ensure sys.path contains the directory that holds the PLD_subsampling/ package directory
if root.name == "PLD_subsampling" and (root / "__init__.py").exists():
    # Notebook opened from inside the package dir → add parent
    sys.path.insert(0, str(root.parent))
elif (root / "PLD_subsampling").exists():
    # Notebook opened from project root → add root
    sys.path.insert(0, str(root))
elif (root.parent / "PLD_subsampling").exists():
    sys.path.insert(0, str(root.parent))

import numpy as np
import matplotlib.pyplot as plt
from dp_accounting.pld import privacy_loss_distribution

from PLD_subsampling.testing.test_utils import run_multiple_experiments, run_experiment
from PLD_subsampling.testing.plot_utils import create_pmf_cdf_plot, create_epsilon_delta_plot, print_experiment_table
from PLD_subsampling.testing.analytic_Gaussian import Gaussian_epsilon_for_delta
from PLD_subsampling.wrappers.dp_accounting_wrappers import amplify_pld_separate_directions
from PLD_subsampling.wrappers.dp_accounting_wrappers import scale_pld_infinity_mass

# Run a single experiment ($\sigma=0.5$, $\lambda=0.1$, remove direction)

In [ ]:
sigma = 0.5
q = 0.1
discretization = 1e-4
deltas = np.array([10 ** (-k) for k in range(2, 13)], dtype=float)
remove_direction = True
versions = run_experiment(sigma=sigma, sampling_prob=q, discretization=discretization, delta_values=deltas, remove_direction=remove_direction)

In [ ]:
print(f"\nσ={sigma}, q={q}, disc={discretization:g}, dir={'rem' if remove_direction else 'add'}")
eps_GT = [
    Gaussian_epsilon_for_delta(sigma=sigma, sampling_prob=q, delta=float(d), remove_direction=remove_direction)
    for d in deltas
]
print_experiment_table(deltas, versions, eps_GT)

dir_text = 'rem' if remove_direction else 'add'
fig_cdf = create_pmf_cdf_plot(versions=versions, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir={dir_text}')
fig_cdf.show()

In [ ]:
fig_eps = create_epsilon_delta_plot(delta_values=deltas, versions=versions, eps_GT=eps_GT, log_x_axis=True, log_y_axis=False, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir:{dir_text}')
fig_eps.show()

# Run many experiments  ($\sigma=2.0$, $\lambda=0.5$)

In [ ]:
discretizations = [1e-4]
q_values = [0.5]
sigma_values = [2.0]
remove_directions = [True, False]
delta_values_arr = np.array([10 ** (-k) for k in range(2, 13)], dtype=float)
results = run_multiple_experiments(discretizations, q_values, sigma_values, remove_directions, delta_values_arr)

### Results for remove direction

In [ ]:
res = results[0]
sigma = res['sigma']; q = res['q']; discretization = res['discretization']; remove_direction = res['remove_direction']
versions = res['versions']; deltas = res['delta_values']
print(f"\nσ={sigma}, q={q}, disc={discretization:g}, dir={'rem' if remove_direction else 'add'}")
eps_GT = [
    Gaussian_epsilon_for_delta(sigma=sigma, sampling_prob=q, delta=float(d), remove_direction=remove_direction)
    for d in deltas
]
print_experiment_table(deltas, versions, eps_GT)

dir_text = 'rem' if remove_direction else 'add'
fig_cdf = create_pmf_cdf_plot(versions=versions, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir={dir_text}')
fig_cdf.show()

### Results for add direction

In [ ]:
res = results[1]
sigma = res['sigma']; q = res['q']; discretization = res['discretization']; remove_direction = res['remove_direction']
versions = res['versions']; deltas = res['delta_values']
print(f"\nσ={sigma}, q={q}, disc={discretization:g}, dir={'rem' if remove_direction else 'add'}")
eps_GT = [
    Gaussian_epsilon_for_delta(sigma=sigma, sampling_prob=q, delta=float(d), remove_direction=remove_direction)
    for d in deltas
]
print_experiment_table(deltas, versions, eps_GT)

dir_text = 'rem' if remove_direction else 'add'
fig_cdf = create_pmf_cdf_plot(versions=versions, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir={dir_text}')
fig_cdf.show()

# Transform a dp_accounting PLD object

In [ ]:
# Correct PLD→PLD transformation example (works without private attributes)
sigma = 1.0
q = 0.05
discretization = 1e-4

# Build a base (non-amplified) PrivacyLossDistribution
base_pld = privacy_loss_distribution.from_gaussian_mechanism(
    standard_deviation=sigma,
    sensitivity=1.0,
    value_discretization_interval=discretization,
    pessimistic_estimate=True,
)
# Amplify separately for remove/add
amplified_pld = amplify_pld_separate_directions(base_pld=base_pld, sampling_prob=q)
# Library-based transformation
lib_amplified_pld =  privacy_loss_distribution.from_gaussian_mechanism(
    standard_deviation=sigma,
    sensitivity=1.0,
    value_discretization_interval=discretization,
    sampling_prob=q,
    pessimistic_estimate=True,
)

#Compute epsilon for the delta array using the two amplified PLDs
eps_GT = [
    Gaussian_epsilon_for_delta(sigma=sigma, sampling_prob=q, delta=float(d), remove_direction=True)
    for d in deltas
]
eps_our_rem = [
    amplified_pld._pmf_remove.get_epsilon_for_delta(d)
    for d in deltas
]
eps_our_add = [
    amplified_pld._pmf_add.get_epsilon_for_delta(d)
    for d in deltas
]
eps_our = np.maximum(eps_our_rem, eps_our_add)
eps_lib = [
    lib_amplified_pld.get_epsilon_for_delta(d)
    for d in deltas
]

versions = [
    {'name': 'Our', 'eps': eps_our},
    {'name': 'Library', 'eps': eps_lib}
]

print_experiment_table(deltas, versions, eps_GT)

plt.plot(deltas, eps_GT, label='GT', alpha=0.5, linestyle='--')
plt.plot(deltas, eps_our, label='Our', alpha=0.5, linestyle='-')
plt.plot(deltas, eps_lib, label='Library', alpha=0.5, linestyle='--')
plt.xscale('log')
plt.legend()
plt.show()

# Increase infinity mass probability of a PLD

In [ ]:
# Compare delta(epsilon) using a fresh PLD (no reuse)
sigma = 1.0
q = 0.05
discretization = 1e-4

epsilons = np.linspace(0.05, 3.0, 20)
added_delta = 1e-4

pld = privacy_loss_distribution.from_gaussian_mechanism(
    standard_deviation=sigma,
    sensitivity=1.0,
    value_discretization_interval=discretization,
    sampling_prob=q,
    pessimistic_estimate=True,
)
pld_scaled = scale_pld_infinity_mass(pld, added_delta)

# Compute delta(ε) using PLD API
delta_before = np.array([pld.get_delta_for_epsilon(float(e)) for e in epsilons])
delta_after = np.array([pld_scaled.get_delta_for_epsilon(float(e)) for e in epsilons])

# Plot
plt.figure(figsize=(6,4))
plt.plot(epsilons, delta_before, label='delta_before', lw=2)
plt.plot(epsilons, delta_after, label='delta_after', lw=2, linestyle='--')
plt.yscale('log')
plt.xlabel('epsilon')
plt.ylabel('delta(epsilon)')
plt.title('PLD: Delta(epsilon) before vs after scaling infinity mass (fresh PLD)')
plt.legend(); plt.grid(True, which='both', ls=':'); plt.show()
